In [2]:
import dagshub
import mlflow
from mlflow.client import MlflowClient
client = MlflowClient(tracking_uri='https://dagshub.com/Raj-Narayanan-B/StudentMLProjectRegression.mlflow',
                      registry_uri='https://dagshub.com/Raj-Narayanan-B/StudentMLProjectRegression.mlflow')
# client = MlflowClient()
import pandas as pd
import numpy as np
import yaml
import os
import json
from pathlib import Path #type: ignore

from src.utils import load_yaml,params_evaluator, load_yaml

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, BaggingClassifier, ExtraTreesClassifier, 
                              HistGradientBoostingClassifier, StackingClassifier, VotingClassifier)
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import hyperopt
from hyperopt.pyll.base import scope
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK, space_eval

os.chdir('f:\\iNeuron\\Projects\\Scania_Truck_Failures')

import mlflow.sklearn
import mlflow.xgboost
import mlflow.pyfunc

f:\iNeuron\Projects\scania_failures_2\scania_truck\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/Raj-Narayanan-B/StudentMLProjectRegression.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'Raj-Narayanan-B'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '8af4cc66be8aec751397fd525e47ae395fa67442'

In [3]:
mlflow.get_tracking_uri()

'https://dagshub.com/Raj-Narayanan-B/StudentMLProjectRegression.mlflow'

In [17]:
mlflow.get_artifact_uri()

[2024-02-26 10:51:25,913: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /Raj-Narayanan-B/StudentMLProjectRegression.mlflow/api/2.0/mlflow/runs/create]


RestException: INVALID_PARAMETER_VALUE: Response: {'error_code': 'INVALID_PARAMETER_VALUE'}

In [ ]:
client.list_artifacts('c03a395f2448493aa7f2f5c4e066e0d3')

In [ ]:
run_id = '7650b7097cb644d08b8fe61bb3065b15'
filter_string=f"tags.run_type ILIKE 'child'"
runs_df = mlflow.search_runs(experiment_ids=['60'])

In [ ]:
runs_df['artifact_uri'][0]

In [ ]:
best_run_id = mlflow.search_runs(experiment_ids=['60'],
                        filter_string=filter_string,
                        order_by = ['metrics.Cost'])[['run_id','artifact_uri','metrics.Cost']]['run_id'][0]
best_run_id

In [ ]:
mlflow.get_run(f'{best_run_id}')

In [ ]:
artifact_path_name = json.loads(mlflow.get_run(f'{best_run_id}').data.tags['mlflow.log-model.history'])[0]['artifact_path']

In [10]:
train_df = pd.read_csv(r"F:\iNeuron\Projects\scania_failures_2\artifacts\data\processed\stage_3_final_processing\processed_train_data.csv").iloc[:2000,:]
test_df = pd.read_csv(r"F:\iNeuron\Projects\scania_failures_2\artifacts\data\processed\stage_3_final_processing\processed_test_data.csv").iloc[:2000,:]

In [11]:
# x = df.drop(columns='class')
# y = df['class']
# # skf = StratifiedKFold()
# x_train,x_test,y_train,y_test = train_test_split(x,y, test_size = 0.25, random_state=8)
x_train,y_train = train_df.drop(columns='class'), train_df['class']
x_test,y_test = test_df.drop(columns='class'), test_df['class']

print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)
print(y_train.value_counts())
print(y_test.value_counts())

(2000, 161) (2000, 161) (2000,) (2000,)
class
0    1973
1      27
Name: count, dtype: int64
class
0    1971
1      29
Name: count, dtype: int64


In [ ]:
params = {'hidden_layer_sizes': (500, 300, 200, 150)}

In [ ]:
np.array(params['hidden_layer_sizes'])

In [ ]:
{"hidden_layer_sizes": np.array(params['hidden_layer_sizes'])}

In [ ]:
mlflow.models.infer_signature(x_train, y_test, {"hidden_layer_sizes": np.array(params['hidden_layer_sizes'])})

In [ ]:
params

In [ ]:
mlflow.log_params({"hidden_layer_sizes": [500, 300, 200, 150]})

In [ ]:
mlflow.set_tracking_uri()

In [ ]:
space = {'n_estimators': scope.int(hp.quniform('n_estimators', 100, 150, 1)),
    'criterion': hp.choice('criterion', ['log_loss', 'entropy', 'gini']),
    'max_features': hp.choice('max_features', ['sqrt', 'log2', None]),
    'class_weight': hp.choice('class_weight', ['balanced', 'balanced_subsample'])}

mlflow.start_run()

mlflow.set_registry_uri
def objective(space):
    hp_rand_forest = RandomForestClassifier(**space)
    hp_rand_forest.fit(x_train,y_train)
    y_pred = hp_rand_forest.predict(x_test)
    tn, fp, fn, tp = confusion_matrix(y_true=y_test, y_pred=y_pred).ravel()
    cost = float((10*fp)+(500*fn))
    return cost

trials = Trials()
best = fmin(fn= objective,
            space= space,
            algo= tpe.suggest,
            max_evals = 1,
            trials= trials)
best_randf=space_eval(space,best)
best_randf

In [ ]:
import mlflow
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

# Create base classifiers
base_classifiers = [
    ('rf', RandomForestClassifier(n_estimators=50)),
    ('gb', GradientBoostingClassifier(n_estimators=50))
]

# Create stacking classifier
stacking_clf = StackingClassifier(
    estimators=base_classifiers,
    final_estimator=DecisionTreeClassifier()
)

# Log parameters
with mlflow.start_run():
    mlflow.log_params({
        'base_classifiers': base_classifiers,
        'final_estimator': 'DecisionTreeClassifier'
    })

    # Train and log other metrics as needed
    # ...

# Close the MLflow run
mlflow.end_run()


In [ ]:
eval('RandomForestClassifier()')

In [ ]:
base_classifiers = [
    ('rf', RandomForestClassifier(n_estimators=50)),
    ('gb', GradientBoostingClassifier(n_estimators=50))
]

params = {'be1':['rf1','RandomForestClassifier()']}

mlflow.models.infer_signature(x_train, y_test, params)

In [ ]:
with mlflow.start_run(experiment_id = 75):
    mlflow.log_param(key = 'be1',
                     value = {'rf1','RandomForestClassifier()'})

In [ ]:
base_classifiers = [
    ('rf', RandomForestClassifier(n_estimators=50)),
    ('gb', GradientBoostingClassifier(n_estimators=50))
]

# Create stacking classifier
stacking_clf = StackingClassifier(
    estimators=base_classifiers,
    final_estimator=DecisionTreeClassifier()
)

# # Log parameters
# with mlflow.start_run(experiment_id = 75):
#     mlflow.log_params({
#         'base_classifiers': eval('base_classifiers'),
#         'final_estimator': eval('DecisionTreeClassifier()')
#     })

In [ ]:
params = {
        'base_classifiers': eval('base_classifiers'),
        'final_estimator': eval('DecisionTreeClassifier()')
    }

In [ ]:
for key,value in params.items():
    params[key] = str(value)

params

In [ ]:
mlflow.models.infer_signature(x_train, y_test, new_params)

In [ ]:
base_classifiers = [
    ('rf', RandomForestClassifier(n_estimators=50)),
    ('gb', GradientBoostingClassifier(n_estimators=50))
]
str(eval('base_classifiers'))

In [ ]:
eval('base_classifiers')

In [ ]:
new_params = client.get_run('ad5d40a9fc33487f8e616dde699ef7ec').data.params

In [ ]:
new_params

In [ ]:
eval("[('KNN_Classifier', KNeighborsClassifier(algorithm='kd_tree', n_neighbors=8))]")

In [ ]:
for key,value in new_params.items():
                try:
                    new_params[key] = eval(value)
                except:
                    new_params[key] = value
                    if value == 'nan':
                        new_params[key] = np.nan
new_params                        

In [ ]:
processed_new_params = {key: value for key, value in new_params.items() if value is not None}

In [ ]:
processed_new_params

In [ ]:
stacking_clf_params = stacking_clf.get_params()
for key, value in stacking_clf_params.items():
    stacking_clf_params[key] = str(value)
stacking_clf_params

In [ ]:
exp_id_list = ['84','85','86']
mlflow.search_runs(experiment_ids = exp_id_list,
                            search_all_experiments = True,
                            filter_string = f"tags.run_type ilike 'parent'")

In [ ]:
client.tracking_uri

In [8]:
client.tracking_uri

'file:///f:/iNeuron/Projects/scania_failures_2/notebooks/mlruns'

In [18]:
models = {'Logistic_Regression': LogisticRegression, 
                  'SGD_Classifier': SGDClassifier,
                  'Random Forest': RandomForestClassifier, 
                  'Ada_Boost': AdaBoostClassifier, 
                  'Grad_Boost': GradientBoostingClassifier, 
                  'Bagging_Classifier': BaggingClassifier, 
                  'ExtraTreesClassifier': ExtraTreesClassifier, 
                  'Hist_Grad_Boost_Classifier': HistGradientBoostingClassifier, 
                  'Decision_Tree_Classifier': DecisionTreeClassifier,
                  'XGB_Classifier': XGBClassifier,
                  'KNN_Classifier': KNeighborsClassifier,
                  'Stacked_Classifier' : StackingClassifier,
                  'Voting_Classifier' : VotingClassifier,
                  'Light_GBM': LGBMClassifier,
                  'LGBMClassifier':""
                  }

for key in models.keys():
    try:
        client.delete_registered_model(f"Candidate_{key}")
    except:
        try:
            client.delete_registered_model(f"Champion {key}")
        except:
            try:
                client.delete_registered_model(f"Challenger {key}")
            except:
                pass

[2024-02-26 09:17:34,107: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /Raj-Narayanan-B/StudentMLProjectRegression.mlflow/api/2.0/mlflow/registered-models/delete]


In [19]:
for i in range(662,676):
    try:
        mlflow.delete_experiment(i)
        print(i)
    except:
        pass

662
663
664
665
666
667
668
669
670
671
672
673
674
675


In [16]:
mlflow.get_experiment_by_name(name ='Voting_classifier')

In [12]:
import re

input_string = '301_Voting_Classifier_301'
match = re.search(r'\d+', input_string)
if match:
    result = match.group()
    print(result)

301


In [ ]:
mlflow.delete_experiment('136389952941582973')

In [ ]:
mlflow.delete_experiment('647179760550514699')

In [ ]:
mlflow.delete_experiment('297515542365458558')

In [ ]:
mlflow.get_tracking_uri()

In [ ]:
for i in range(1,106):
    try:
        os.system(f'mlflow gc --experiment-ids {i}')
        print(i)
    except:
        pass

In [ ]:
[i() for i in models.values()]

In [ ]:
models = {'Logistic_Regression': LogisticRegression, 
                  'SGD_Classifier': SGDClassifier,
                  'Random Forest': RandomForestClassifier, 
                  'Ada_Boost': AdaBoostClassifier, 
                  'Grad_Boost': GradientBoostingClassifier, 
                  'Bagging_Classifier': BaggingClassifier, 
                  'ExtraTreesClassifier': ExtraTreesClassifier, 
                  'Hist_Grad_Boost_Classifier': HistGradientBoostingClassifier, 
                  'Decision_Tree_Classifier': DecisionTreeClassifier,
                  # 'XGB_Classifier': XGBClassifier,
                  'KNN_Classifier': KNeighborsClassifier,
                #   'Stacked_Classifier' : StackingClassifier,
                #   'Voting_Classifier' : VotingClassifier
                  }
estimators = list(zip(models.keys(),[i() for i in models.values()]))
estimators

In [ ]:
stacking_clf = StackingClassifier(estimators = estimators,
                                  final_estimator = XGBClassifier(),
                                  stack_method = 'auto',
                                  passthrough=True)

In [ ]:
stacking_clf.get_params()

In [ ]:
client.tracking_uri

In [ ]:
client._registry_uri

In [ ]:
stacked_clf_params = stacking_clf.get_params()

In [ ]:
with mlflow.start_run(experiment_id = 141):
    mlflow.log_params({key: value for key, value in stacked_clf_params.items() if value is not None})

In [ ]:
{key: value for key, value in stacked_clf_params.items() if value is not None}

In [ ]:
models = {'Logistic_Regression': LogisticRegression, 
                  'SGD_Classifier': SGDClassifier,
                  'Random Forest': RandomForestClassifier, 
                  'Ada_Boost': AdaBoostClassifier, 
                  'Grad_Boost': GradientBoostingClassifier, 
                  'Bagging_Classifier': BaggingClassifier, 
                  'ExtraTreesClassifier': ExtraTreesClassifier, 
                  'Hist_Grad_Boost_Classifier': HistGradientBoostingClassifier, 
                  'Decision_Tree_Classifier': DecisionTreeClassifier,
                  'XGB_Classifier': XGBClassifier,
                  'KNN_Classifier': KNeighborsClassifier,
                #   'Stacked_Classifier' : StackingClassifier,
                #   'Voting_Classifier' : VotingClassifier
                  }
estimators = list(zip(models.keys(), [i() for i in models.values()]))#[value() if key != 'XGB_Classifier' else 'XGBClassifier()' for key,value in models.items()]))
estimators

In [ ]:
for i in range(len(estimators)):
    if estimators[i][0] == 'XGB_Classifier':
        xgb_params = {key: value for key, value in estimators[i][1].get_params().items() if value is not None}

xgb_params

In [ ]:
xgb_params

In [ ]:
new_str

In [ ]:
estimators

In [ ]:
for i in range(len(estimators)):
    if estimators[i][0] == 'XGB_Classifier':
        estimators[i][1].set_params(**xgb_params)

In [ ]:
str(estimators)

In [ ]:
new_params = {'estimators' : estimators,
              'XGB_Classifier' : 'XGBClassifier()',
              'final_estimator' : 'XGBClassifier()',
              'stack_method' : 'auto',
              'passthrough' : 'True',
              'xgb_params': xgb_params}

In [ ]:
new_params

In [ ]:
with mlflow.start_run(experiment_id = 141):
    mlflow.log_params(new_params)
    # mlflow.log_param(key = 'XGBClassifier',
    #                  value = XGBClassifier())
    # mlflow.log_param(key = 'final_estimator',
    #                  value = XGBClassifier())

In [ ]:
stacked_run = mlflow.get_run('aaaec579ec5f4a63bad4dd03a8c6f1ae')

In [ ]:
import numpy as np
run_params = {}
for key,value in stacked_run.data.params.items():
    try:
        run_params[key] = eval(value)
    except:
        run_params[key] = value
        if value == 'nan':
                            run_params[key] = np.nan
run_params

In [ ]:
run_params['xgb_params'] = run_params['xgb_params'].replace("nan",'None')
xgb_params_ = eval(run_params['xgb_params'])
xgb_params['missing'] = np.nan
xgb_params

In [ ]:
run_params['estimators'].append(('XGBClassifier',run_params['XGB_Classifier'].set_params(**xgb_params)))

In [ ]:
run_params

In [ ]:
estimators

In [ ]:
s_clf = StackingClassifier(estimators=estimators,
                   final_estimator=XGBClassifier(),
                   cv=5,
                   stack_method = 'auto',
                   passthrough=True)

# estimators,
# final_estimator,
# cv,
# stack_method,
# passthrough
s_clf.get_params()

In [ ]:
s_clf.get_params()['estimators']
s_clf.get_params()['final_estimator']
s_clf.get_params()['cv']
s_clf.get_params()['stack_method']
s_clf.get_params()['passthrough']

In [ ]:
s_clf.get_params()['estimators']

In [ ]:
f"{s_clf.get_params()['final_estimator'].__class__.__name__}_params"

In [ ]:
flag = 0
for i in range(len(s_clf.get_params()['estimators'])):
    if 'XGB_Classifier' == s_clf.get_params()['estimators'][i][0]:
        flag =1
if flag == 1:
    clf_list = []
    for i in range(len(s_clf.get_params()['estimators'])):
        if s_clf.get_params()['estimators'][i][0] != "XGB_Classifier":
            clf_list.append(s_clf.get_params()['estimators'][i])
    estimators_params = {}
    for i in range(len(s_clf.get_params()['estimators'])):
        estimators_params[s_clf.get_params()['estimators'][i][0]] = s_clf.get_params()['estimators'][i][1].get_params()
        if 'XGB_Classifier' == s_clf.get_params()['estimators'][i][0]:
            xgb_params = s_clf.get_params()['estimators'][i][1].get_params()
    xgb_params = {key:value for key,value in xgb_params.items() if value is not None}
    
    if s_clf.get_params()['final_estimator'].__class__.__name__ == 'XGBClassifier':
        s_clf_params = {'estimators' : clf_list,
                'final_estimator' : 'XGBClassifier()',
                'cv' : 8,#s_clf.get_params()['cv'],
                'stack_method' : s_clf.get_params()['stack_method'],
                'passthrough' : s_clf.get_params()['passthrough'],
                'xgb_params' : xgb_params}
    else:
        s_clf_params = {'estimators' : clf_list,
                'final_estimator' : s_clf.get_params()['final_estimator'],
                'cv' : s_clf.get_params()['cv'],
                'stack_method' : s_clf.get_params()['stack_method'],
                'passthrough' : s_clf.get_params()['passthrough'],
                'xgb_params' : xgb_params}

    mlflow.log_params(s_clf_params)
    mlflow.models.infer_signature(model_input=x_train,
                              model_output = s_clf.predict(x_train))
    mlflow.sklearn.log_model()
    

In [ ]:
s_clf.fit(x_train,y_train)

In [ ]:
s_clf_params_ = s_clf.get_params()
for key, value in s_clf_params_.items():
    s_clf_params_[key] = str(value)
signature = mlflow.models.infer_signature(model_input = x_train,
                                          model_output = s_clf.predict(x_train),
                                          params = s_clf_params_)

In [ ]:
signature

In [ ]:
for i in range(len(s_clf.get_params()['estimators'])):
    if 'XGB_Classifier' == s_clf.get_params()['estimators'][i][0]:
        xgb_params = s_clf.get_params()['estimators'][i][1].get_params()
xgb_params = {key:value for key,value in xgb_params.items() if value is not None}
xgb_params

In [ ]:
xgb = XGBClassifier()

In [ ]:
xgb_params = {key:value for key,value in xgb.get_params().items() if value is not None}
xgb_params


In [ ]:
s_clf_params

In [ ]:
s_clf_params = {'estimators' : clf_list,
          'final_estimator' : 'XGBClassifier()',
          'cv' : s_clf.get_params()['cv'],
          'stack_method' : s_clf.get_params()['stack_method'],
          'passthrough' : s_clf.get_params()['passthrough'],
          'xgb_params' : xgb_params}

s_clf_params

In [ ]:
trial_params = {'estimators': [('SGD_Classifier', SGDClassifier(loss='log_loss')), 
                ('Random Forest', RandomForestClassifier(class_weight='balanced_subsample', criterion='entropy', max_features=None, n_estimators=139)), 
                ('Hist_Grad_Boost_Classifier', HistGradientBoostingClassifier(max_iter=575)), 
                ('Bagging_Classifier', BaggingClassifier(n_estimators=59)), 
                ('Ada_Boost', AdaBoostClassifier(algorithm='SAMME', n_estimators=133)), 
                ('Grad_Boost', GradientBoostingClassifier(criterion='squared_error', loss='exponential', max_features='log2', n_estimators=143))], 
 'final_estimator': 'XGBClassifier()', 
 'cv': 5, 
 'stack_method': 'auto', 
 'passthrough': False, 
 'xgb_params': {'objective': 'binary:logistic', 
                'booster': 'dart', 
                'enable_categorical': False, 
                'learning_rate': 6.5697687477009445, 
                'missing': np.nan, 
                'n_estimators': 164, 
                'tree_method': 'hist'}
}

In [ ]:
trial_params['estimators'].append(('XGB_Classifier',XGBClassifier()))
trial_params

In [ ]:
trial_params['estimators']

In [ ]:
s_clf.get_params()['final_estimator']

In [ ]:
clf_list = []
for i in range(len(trial_params['estimators'])):
    clf_list.append((trial_params['estimators'][i][0],trial_params['estimators'][i][1].__class__()))

estimators_params = {}
for i in range(len(trial_params['estimators'])):
        estimators_params[trial_params['estimators'][i][0]] = trial_params['estimators'][i][1].get_params()
for i in estimators_params:
        estimators_params[i] = {key:value for key,value in estimators_params[i].items() if value is not None}
s_clf_params_trial_2 = {'estimators' : clf_list,
                'final_estimator' : s_clf.get_params()['final_estimator'],
                'cv' : s_clf.get_params()['cv'],
                'stack_method' : s_clf.get_params()['stack_method'],
                'passthrough' : s_clf.get_params()['passthrough']}
for key,value in estimators_params.items():
        s_clf_params_trial_2[f"{key}_params"] = value
with mlflow.start_run(experiment_id = 154):
    mlflow.log_params(s_clf_params_trial_2)

In [ ]:
with mlflow.start_run(experiment_id = 154):
    mlflow.log_params(s_clf_params_trial)

In [ ]:
s_clf = StackingClassifier(estimators = trial_params['estimators'],
                           final_estimator=RandomForestClassifier(class_weight='balanced_subsample', criterion='entropy',
                         max_features=None, n_estimators=139),
                           cv=5,
                           stack_method = 'auto',
                           passthrough=True)

In [ ]:
s_clf

In [ ]:
s_clf.get_params()['estimators'][5][1].__class__()

In [ ]:
flag = 0
for i in range(len(s_clf.get_params()['estimators'])):
    if 'XGB_Classifier' == s_clf.get_params()['estimators'][i][0]:
        flag =1
if flag == 1:
    clf_list = []
    for i in range(len(s_clf.get_params()['estimators'])):
        if s_clf.get_params()['estimators'][i][0] != "XGB_Classifier":
            clf_list.append((s_clf.get_params()['estimators'][i][0],s_clf.get_params()['estimators'][i][1].__class__()))
    estimators_params = {}
    for i in range(len(s_clf.get_params()['estimators'])):
        estimators_params[s_clf.get_params()['estimators'][i][0]] = s_clf.get_params()['estimators'][i][1].get_params()

    for i in estimators_params:
        estimators_params[i] = {key:value for key,value in estimators_params[i].items() if value is not None}
    
    if s_clf.get_params()['final_estimator'].__class__.__name__ == 'XGBClassifier':
        
            s_clf_params = {'estimators' : clf_list,
                    'final_estimator' : 'XGBClassifier()',
                    'cv' : s_clf.get_params()['cv'],
                    'stack_method' : s_clf.get_params()['stack_method'],
                    'passthrough' : s_clf.get_params()['passthrough']}
            for key,value in estimators_params.items():
                s_clf_params[f"{key}_params"] = value

    else:
        s_clf_params = {'estimators' : clf_list,
                'final_estimator' : s_clf.get_params()['final_estimator'],
                'cv' : s_clf.get_params()['cv'],
                'stack_method' : s_clf.get_params()['stack_method'],
                'passthrough' : s_clf.get_params()['passthrough']}
        for key,value in estimators_params.items():
                s_clf_params[f"{key}_params"] = value

with mlflow.start_run(experiment_id = 154):
    mlflow.log_params(s_clf_params)
#     # mlflow.models.infer_signature(model_input=x_train,
#     #                           model_output = s_clf.predict(x_train))
#     # mlflow.sklearn.log_model()
    

In [ ]:
s_clf_params

In [ ]:
s_clf.get_params()['final_estimator']

In [ ]:
s_clf_params

In [ ]:
s_clf.get_params()['estimators'][0][0]

In [ ]:
estimators_params

In [ ]:
s_clf.get_params()['final_estimator'].__class__.__name__ == 'XGBClassifier'

In [ ]:
name_of_dictionary = "my_dict"
globals()[name_of_dictionary] = {'hi':1}

In [ ]:
name_of_dictionary

In [ ]:
estimators_params['XGB_Classifier'].items()

In [ ]:
with mlflow.start_run(experiment_id = 154):
    mlflow.log_params(trial_params)

In [ ]:
mlflow.models.infer_signature(model_input=x_train,
                              model_output = s_clf.predict(x_train))

In [ ]:
v_clf = VotingClassifier()

In [ ]:
s_clf_run = client.get_run('3f360d91251446b488d0255f44ea1658')

In [ ]:
best_model = mlflow.sklearn.load_model('mlflow-artifacts:/11856c6b7da848c38f85aa3ece46ad6c/3f360d91251446b488d0255f44ea1658/artifacts/optuna_Stacked_Classifier')

In [ ]:
best_model.get_params()

In [ ]:
s_clf_run.data.params

In [ ]:
best_model1 = mlflow.sklearn.load_model('mlflow-artifacts:/11856c6b7da848c38f85aa3ece46ad6c/5c209d92a1174a5aaefc84258e073304/artifacts/optuna_Stacked_Classifier')
best_model1.get_params()

In [ ]:
import mlflow.xgboost

In [ ]:
xgboost = mlflow.xgboost.load_model('mlflow-artifacts:/1cd799950ea74cae9dbec616b6b65993/acc08000975c49d7a3818110253855f1/artifacts/optuna_XGB_Classifier')

In [ ]:
params = client.get_run('acc08000975c49d7a3818110253855f1').data.params

In [ ]:
params

In [ ]:
with mlflow.start_run(experiment_id='188'):
    mlflow.log_params(params)

In [ ]:
params_ = client.get_run('13ba590fba3f4584ba34cf81006dc31d').data.params

In [ ]:
params_

In [ ]:
rand_f_model = mlflow.sklearn.load_model("mlflow-artifacts:/489d8cb8991a4e58b57565bc00e31b9e/34cc0da17feb4e0ea750551a65809ba8/artifacts/optuna_Random Forest")

In [ ]:
rand_f_model.get_params()

In [ ]:
rand_f_params = client.get_run('34cc0da17feb4e0ea750551a65809ba8').data.params
rand_f_params

In [ ]:
with mlflow.start_run(experiment_id='181'):
    mlflow.log_params(rand_f_params)

In [10]:
exp_id = mlflow.create_experiment(name = 'test',
                                  artifact_location = 'sample_test')
with mlflow.start_run(#experiment_id=exp_id,
                      run_id = 56):
    

RestException: BAD_REQUEST: Response: {'error_code': 'BAD_REQUEST'}

In [14]:
from src.constants import PARAMS_PATH


params = load_yaml(r'F:\iNeuron\Projects\scania_failures_2\params.yaml')

[2024-02-08 21:57:38,113: INFO: utils: params.yaml yaml_file is loaded]


In [71]:
from pprint import pprint
pprint(dict(params['models']), compact = True)

{'Ada_Boost': 'AdaBoostClassifier',
 'Bagging_Classifier': 'BaggingClassifier',
 'Decision_Tree_Classifier': 'DecisionTreeClassifier',
 'ExtraTreesClassifier': 'ExtraTreesClassifier',
 'Grad_Boost': 'GradientBoostingClassifier',
 'Hist_Grad_Boost_Classifier': 'HistGradientBoostingClassifier',
 'KNN_Classifier': 'KNeighborsClassifier',
 'Light_GBM': 'LGBMClassifier',
 'Logistic_Regression': 'LogisticRegression',
 'Random Forest': 'RandomForestClassifier',
 'SGD_Classifier': 'SGDClassifier',
 'XGB_Classifier': 'XGBClassifier'}


In [78]:
pprint({key: eval(value) for key,value in params['models'].items()}, compact =True)

{'Ada_Boost': <class 'sklearn.ensemble._weight_boosting.AdaBoostClassifier'>,
 'Bagging_Classifier': <class 'sklearn.ensemble._bagging.BaggingClassifier'>,
 'Decision_Tree_Classifier': <class 'sklearn.tree._classes.DecisionTreeClassifier'>,
 'ExtraTreesClassifier': <class 'sklearn.ensemble._forest.ExtraTreesClassifier'>,
 'Grad_Boost': <class 'sklearn.ensemble._gb.GradientBoostingClassifier'>,
 'Hist_Grad_Boost_Classifier': <class 'sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingClassifier'>,
 'KNN_Classifier': <class 'sklearn.neighbors._classification.KNeighborsClassifier'>,
 'Light_GBM': <class 'lightgbm.sklearn.LGBMClassifier'>,
 'Logistic_Regression': <class 'sklearn.linear_model._logistic.LogisticRegression'>,
 'Random Forest': <class 'sklearn.ensemble._forest.RandomForestClassifier'>,
 'SGD_Classifier': <class 'sklearn.linear_model._stochastic_gradient.SGDClassifier'>,
 'XGB_Classifier': <class 'xgboost.sklearn.XGBClassifier'>}


In [30]:
eval(params.models['Light_GBM'])(**light_gbm_params)

LGBMClassifier(boosting_type='dart', class_weight='balanced',
               learning_rate=0.4440057790393853, n_estimators=133, n_jobs=-1)

In [8]:
from src.utils import params_evaluator
exp_count = 286
light_gbm_run = mlflow.search_runs(experiment_names=[f"{exp_count}_Light_GBM_{exp_count}"],
                                               filter_string="tags.model ilike 'Light_GBM' and tags.run_type ilike 'parent'")['run_id'][0]
# light_gbm_params = params_evaluator(mlflow.get_run(light_gbm_run).data.params)

[2024-02-08 21:38:27,358: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /Raj-Narayanan-B/StudentMLProjectRegression.mlflow/api/2.0/mlflow/experiments/get-by-name?experiment_name=286_Light_GBM_286]


In [25]:
exp_count = 286
mlflow.search_runs(experiment_names=[f"{exp_count}_{model_name}_{exp_count}"],
                                               filter_string=f"tags.model ilike '{model_name}' and tags.run_type ilike 'parent'",
                                               order_by=['metrics.Accuracy_Score DESC'])['tags.model']


NameError: name 'exp_count' is not defined

In [29]:
model_name = "Light_GBM"
exp_count = 286
light_gbm_run = mlflow.search_runs(experiment_names=[f"{exp_count}_{model_name}_{exp_count}"],
                                               filter_string=f"tags.model ilike '{model_name}' and tags.run_type ilike 'parent'",
                                               order_by=['metrics.Accuracy_Score DESC'])['run_id'][0]
light_gbm_params = params_evaluator(mlflow.get_run(light_gbm_run).data.params)
light_gbm_params

{'boosting_type': 'dart',
 'class_weight': 'balanced',
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.4440057790393853,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 133,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [9]:
light_gbm_run

'01a5f1ce8215498aa29d5b85c4dd73c8'

In [31]:
filter_string = None
registered_models = {mlflow.search_registered_models(filter_string=filter_string)[i].latest_versions[0].name: mlflow.search_registered_models(
    filter_string=filter_string)[i].latest_versions[0].run_id for i in range(len(mlflow.search_registered_models(filter_string=filter_string)))}
registered_models

[2024-02-08 22:17:51,503: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /Raj-Narayanan-B/StudentMLProjectRegression.mlflow/api/2.0/mlflow/registered-models/search?max_results=100]


{'Candidate_Bagging_Classifier': '64d2bf5658014835aa049d0f9caef2e7',
 'Candidate_Light_GBM': 'f840116751a0437dac90034ce42658fa'}

In [33]:
from src.utils import params_evaluator
# Get the accuracies and the respective params of the models as dict
run_details = {}
for key, value in registered_models.items():
    run_details[client.get_run(value).data.tags['model']] = {}
    run_details[client.get_run(value).data.tags['model']]['accuracy'] = client.get_run(
        value).data.metrics['Accuracy_Score']
    run_details[client.get_run(value).data.tags['model']]['params'] = params_evaluator(
        client.get_run(value).data.params)
run_details

{'Bagging_Classifier': {'accuracy': 0.983699778651713,
  'params': {'bootstrap': True,
   'bootstrap_features': False,
   'estimator__boosting_type': 'gbdt',
   'estimator__class_weight': 'balanced',
   'estimator__colsample_bytree': 1.0,
   'estimator__importance_type': 'split',
   'estimator__learning_rate': 0.541189837146803,
   'estimator__max_depth': -1,
   'estimator__min_child_samples': 20,
   'estimator__min_child_weight': 0.001,
   'estimator__min_split_gain': 0.0,
   'estimator__n_estimators': 122,
   'estimator__n_jobs': -1,
   'estimator__num_leaves': 31,
   'estimator__objective': None,
   'estimator__random_state': None,
   'estimator__reg_alpha': 0.0,
   'estimator__reg_lambda': 0.0,
   'estimator__subsample': 1.0,
   'estimator__subsample_for_bin': 200000,
   'estimator__subsample_freq': 0,
   'estimator': LGBMClassifier(class_weight='balanced', learning_rate=0.541189837146803,
                  n_estimators=122, n_jobs=-1),
   'max_features': 1.0,
   'max_samples': 1.0

In [34]:

# Create a dataframe from the "run_details" dict and sort it by "accuracy" in DESC
# In this dataframe only the models whose accuracy is greater than 0.9 are chosen.
models_df = pd.DataFrame(run_details).T
models_df

,accuracy,params
Bagging_Classifier,0.9837,"{'bootstrap': True, 'bootstrap_features': Fals..."
Light_GBM,0.9864,"{'boosting_type': 'dart', 'class_weight': 'bal..."


In [58]:
sorted_models_df = models_df[models_df['accuracy'] > 0.9].sort_values(
    by='accuracy', ascending=False)
sorted_models_df

,accuracy,params
Light_GBM,0.9864,"{'boosting_type': 'dart', 'class_weight': 'bal..."
Bagging_Classifier,0.9837,"{'bootstrap': True, 'bootstrap_features': Fals..."


In [37]:
models = {  # 'Logistic_Regression': LogisticRegression,
            #   'SGD_Classifier': SGDClassifier,
            #   'Random Forest': RandomForestClassifier,
            #   'Ada_Boost': AdaBoostClassifier,
            #   'Grad_Boost': GradientBoostingClassifier,
            'Light_GBM': LGBMClassifier,
            'Bagging_Classifier': BaggingClassifier,
            #   'ExtraTreesClassifier': ExtraTreesClassifier,
            #   'Hist_Grad_Boost_Classifier': HistGradientBoostingClassifier,
            #   'Decision_Tree_Classifier': DecisionTreeClassifier,
            #   'XGB_Classifier': XGBClassifier,
            #   'KNN_Classifier': KNeighborsClassifier,
        }
# Using the sorted_models_df from above, we are creating another dict that has the models fitted with the parameters.
mlflow_models = {key: value(**(sorted_models_df.params[key])) for key, value in models.items() if key in sorted_models_df.index}


TypeError: BaggingClassifier.__init__() got an unexpected keyword argument 'estimator__boosting_type'

In [59]:
sorted_models_df.params['Bagging_Classifier']#['n_estimators']

{'bootstrap': True,
 'bootstrap_features': False,
 'estimator__boosting_type': 'gbdt',
 'estimator__class_weight': 'balanced',
 'estimator__colsample_bytree': 1.0,
 'estimator__importance_type': 'split',
 'estimator__learning_rate': 0.541189837146803,
 'estimator__max_depth': -1,
 'estimator__min_child_samples': 20,
 'estimator__min_child_weight': 0.001,
 'estimator__min_split_gain': 0.0,
 'estimator__n_estimators': 122,
 'estimator__n_jobs': -1,
 'estimator__num_leaves': 31,
 'estimator__objective': None,
 'estimator__random_state': None,
 'estimator__reg_alpha': 0.0,
 'estimator__reg_lambda': 0.0,
 'estimator__subsample': 1.0,
 'estimator__subsample_for_bin': 200000,
 'estimator__subsample_freq': 0,
 'estimator': LGBMClassifier(class_weight='balanced', learning_rate=0.541189837146803,
                n_estimators=122, n_jobs=-1),
 'max_features': 1.0,
 'max_samples': 1.0,
 'n_estimators': 68,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': F

In [60]:
mlflow_models = {}
for key, value in models.items():
    if key in sorted_models_df.index:
        if key == 'Bagging_Classifier':
            sorted_models_df.params['Bagging_Classifier'] = {'n_estimators': sorted_models_df.params['Bagging_Classifier']['n_estimators'],
                                                 'n_jobs':sorted_models_df.params['Bagging_Classifier']['n_jobs'],
                                                 'estimator':sorted_models_df.params['Bagging_Classifier']['estimator']}
        mlflow_models[key] = value(**(sorted_models_df.params[key]))

In [61]:
mlflow_models

{'Light_GBM': LGBMClassifier(boosting_type='dart', class_weight='balanced',
                learning_rate=0.4440057790393853, n_estimators=133, n_jobs=-1),
 'Bagging_Classifier': BaggingClassifier(estimator=LGBMClassifier(class_weight='balanced',
                                            learning_rate=0.541189837146803,
                                            n_estimators=122, n_jobs=-1),
                   n_estimators=68)}

In [62]:



# Create the list[tuple] best_estimators to fit in the voting classifier
best_estimators_mlflow = list(
    zip(mlflow_models.keys(), mlflow_models.values()))
best_estimators_mlflow

[('Light_GBM',
  LGBMClassifier(boosting_type='dart', class_weight='balanced',
                 learning_rate=0.4440057790393853, n_estimators=133, n_jobs=-1)),
 ('Bagging_Classifier',
  BaggingClassifier(estimator=LGBMClassifier(class_weight='balanced',
                                             learning_rate=0.541189837146803,
                                             n_estimators=122, n_jobs=-1),
                    n_estimators=68))]

In [64]:

# If using stacking classifier, get the final estimator using:
final_estimator_mlflow = {key: value(**(sorted_models_df.iloc[:1, :].params[key]))
                            for key, value in models.items() if key in sorted_models_df.iloc[:1, :].index}
# Access the final estimator model using:
final_estimator_mlflow

{'Light_GBM': LGBMClassifier(boosting_type='dart', class_weight='balanced',
                learning_rate=0.4440057790393853, n_estimators=133, n_jobs=-1)}

In [65]:

final_estimator = final_estimator_mlflow[sorted_models_df.iloc[:1, :].index.values[0]]
final_estimator

LGBMClassifier(boosting_type='dart', class_weight='balanced',
               learning_rate=0.4440057790393853, n_estimators=133, n_jobs=-1)

In [81]:
{key: eval(value) for key, value in params['models'].items()}

{'Logistic_Regression': <class 'sklearn.linear_model._logistic.LogisticRegression'>, 'SGD_Classifier': <class 'sklearn.linear_model._stochastic_gradient.SGDClassifier'>, 'Random Forest': <class 'sklearn.ensemble._forest.RandomForestClassifier'>, 'Ada_Boost': <class 'sklearn.ensemble._weight_boosting.AdaBoostClassifier'>, 'Grad_Boost': <class 'sklearn.ensemble._gb.GradientBoostingClassifier'>, 'Light_GBM': <class 'lightgbm.sklearn.LGBMClassifier'>, 'Bagging_Classifier': <class 'sklearn.ensemble._bagging.BaggingClassifier'>, 'ExtraTreesClassifier': <class 'sklearn.ensemble._forest.ExtraTreesClassifier'>, 'Hist_Grad_Boost_Classifier': <class 'sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingClassifier'>, 'Decision_Tree_Classifier': <class 'sklearn.tree._classes.DecisionTreeClassifier'>, 'XGB_Classifier': <class 'xgboost.sklearn.XGBClassifier'>, 'KNN_Classifier': <class 'sklearn.neighbors._classification.KNeighborsClassifier'>}

In [79]:
voter_clf = mlflow.sklearn.load_model('mlflow-artifacts:/26beeea305ba477ab9dccccd03b1c107/7b0b26910f2244de906c9fee702e2717/artifacts/candidate_Voting_Classifier')

[2024-02-09 00:02:01,152: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /Raj-Narayanan-B/StudentMLProjectRegression.mlflow/api/2.0/mlflow-artifacts/artifacts?path=26beeea305ba477ab9dccccd03b1c107%2F7b0b26910f2244de906c9fee702e2717%2Fartifacts%2Fcandidate_Voting_Classifier]


[2024-02-09 00:02:21,438: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /Raj-Narayanan-B/StudentMLProjectRegression.mlflow/api/2.0/mlflow-artifacts/artifacts/26beeea305ba477ab9dccccd03b1c107/7b0b26910f2244de906c9fee702e2717/artifacts/candidate_Voting_Classifier/conda.yaml]
[2024-02-09 00:02:21,451: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /Raj-Narayanan-B/StudentMLProjectRegression.mlflow/api/2.0/mlflow-artifacts/artifacts/26beeea305ba477ab9dccccd03b1c107/7b0b26910f2244de906c9fee702e2717/artifacts/candidate_Voting_Classifier/model.pkl]
[2024-02-09 00:02:21,458: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected(

In [80]:
voter_clf

VotingClassifier(estimators=[('Light_GBM',
                              LGBMClassifier(boosting_type='dart',
                                             class_weight='balanced',
                                             learning_rate=0.38763535810228855,
                                             n_estimators=115, n_jobs=-1)),
                             ('Bagging_Classifier',
                              BaggingClassifier(estimator=LGBMClassifier(boosting_type='dart',
                                                                         class_weight='balanced',
                                                                         learning_rate=0.38763535810228855,
                                                                         n_estimators=115,
                                                                         n_jobs=-1),
                                                n_estimators=60, n_jobs=-1))],
                 n_jobs=-1, verbose=True)

In [1]:
len(['aa_000', 'ac_000', 'ad_000', 'ae_000', 'af_000', 'ag_000', 'ag_001', 'ag_002', 'ag_003', 'ag_004', 'ag_005', 'ag_006', 'ag_007', 'ag_008', 'ag_009', 'ah_000', 'ai_000', 'aj_000', 'ak_000', 'al_000', 'am_0', 'an_000', 'ao_000', 'ap_000', 'aq_000', 'ar_000', 'as_000', 'at_000', 'au_000', 'av_000', 'ax_000', 'ay_000', 'ay_001', 'ay_002', 'ay_003', 'ay_004', 'ay_005', 'ay_006', 'ay_007', 'ay_008', 'ay_009', 'az_000', 'az_001', 'az_002', 'az_003', 'az_004', 'az_005', 'az_006', 'az_007', 'az_008', 'az_009', 'ba_000', 'ba_001', 'ba_002', 'ba_003', 'ba_004', 'ba_005', 'ba_006', 'ba_007', 'ba_008', 'ba_009', 'bb_000', 'bc_000', 'bd_000', 'be_000', 'bf_000', 'bg_000', 'bh_000', 'bi_000', 'bj_000', 'bk_000', 'bl_000', 'bs_000', 'bt_000', 'bu_000', 'bv_000', 'bx_000', 'by_000', 'bz_000', 'ca_000', 'cb_000', 'cc_000', 'ce_000', 'cf_000', 'cg_000', 'ch_000', 'ci_000', 'cj_000', 'ck_000', 'cl_000', 'cm_000', 'cn_000', 'cn_001', 'cn_002', 'cn_003', 'cn_004', 'cn_005', 'cn_006', 'cn_007', 'cn_008', 'cn_009', 'co_000', 'cp_000', 'cq_000', 'cs_000', 'cs_001', 'cs_002', 'cs_003', 'cs_004', 'cs_005', 'cs_006', 'cs_007', 'cs_008', 'cs_009', 'ct_000', 'cu_000', 'cv_000', 'cx_000', 'cy_000', 'cz_000', 'da_000', 'db_000', 'dc_000', 'dd_000', 'de_000', 'df_000', 'dg_000', 'dh_000', 'di_000', 'dj_000', 'dk_000', 'dl_000', 'dm_000', 'dn_000', 'do_000', 'dp_000', 'dq_000', 'dr_000', 'ds_000', 'dt_000', 'du_000', 'dv_000', 'dx_000', 'dy_000', 'dz_000', 'ea_000', 'eb_000', 'ec_00', 'ed_000', 'ee_000', 'ee_001', 
'ee_002', 'ee_003', 'ee_004', 'ee_005', 'ee_006', 'ee_007', 'ee_008', 'ee_009', 'ef_000', 'eg_000'])

161